# Plotting Interpolation for ControlNet

In [1]:
# Modules
import torch

import os

from PIL import Image

In [2]:
image_type = "cat"
image_path = "sample_results"
methods = ["ProbGEORCE_D", "ProbGEORCE_N", "linear", "slerp", "noisediffusion", "noise"]

## Loading Images

In [3]:
img1 = Image.open(f'sample_imgs/{image_type}1.png').resize((768, 768))
img2 = Image.open(f'sample_imgs/{image_type}2.png').resize((768, 768))

In [4]:
save_path = ''.join((image_path, f'/{image_type}', '/'))    
onlyfiles = [f for f in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, f))]

In [5]:
onlyfiles

['000.png', 'args.yaml', '002.png']